# CNN Fashion Style Classification Modeling



## Import Library

In [1]:
import tensorflow as tf
import numpy as np

from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.applications import vgg16

from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Conv2D, MaxPooling2D, Flatten, BatchNormalization, Dropout
from tensorflow.keras.models import Model, Sequential

## Import Data

In [2]:
tr = '/content/drive/MyDrive/style/tr'
val = '/content/drive/MyDrive/style/val'

# 클래스 리스트 선언
class_list = ['street', 'simple', 'classic', 'work', 'unique', 'sexy', 'girlish']

tr = tf.keras.preprocessing.image_dataset_from_directory(
    tr,
    labels="inferred",
    label_mode="categorical",
    class_names=class_list,
    seed=6,
)

val = tf.keras.preprocessing.image_dataset_from_directory(
    val,
    labels="inferred",
    label_mode="categorical",
    class_names=class_list,
    seed=6,
)

tr, val

NotFoundError: Could not find directory /content/drive/MyDrive/style/tr

## Baseline Model

In [ ]:
# 간단한 CNN 신경망 구축
model1 = Sequential()
model1.add(Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)))
model1.add(MaxPooling2D((2, 2)))
model1.add(Conv2D(64, (3, 3), activation='relu'))
model1.add(MaxPooling2D((2, 2)))
model1.add(Conv2D(64, (3, 3), activation='relu'))
model1.add(Flatten())
model1.add(Dense(64, activation='relu'))
model1.add(Dense(7, activation='softmax'))

model1.summary()

In [ ]:
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model1.fit(tr, validation_data=val, epochs=5)

model1.evaluate(val)

## Model 개선

In [ ]:
model2 = Sequential()
model2.add(Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same', 
                    activation ='relu', input_shape = (256, 256, 3)))
model2.add(Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same', 
                    activation ='relu'))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(BatchNormalization())
model2.add(Dropout(0.25))
model2.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                    activation ='relu'))
model2.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                    activation ='relu'))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(BatchNormalization())
model2.add(Dropout(0.25))
model2.add(Conv2D(filters = 86, kernel_size = (3,3),padding = 'Same', 
                    activation ='relu'))
model2.add(Conv2D(filters = 86, kernel_size = (3,3),padding = 'Same', 
                    activation ='relu'))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(BatchNormalization())
model2.add(Dropout(0.25))
model2.add(Flatten())
model2.add(Dense(1024, activation = "relu"))
model2.add(Dropout(0.5))
model2.add(Dense(512, activation = "relu"))
model2.add(Dropout(0.5))
model2.add(Dense(7, activation = "softmax"))
model2.summary()

In [ ]:
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model2.fit(tr, validation_data=val, epochs=10)

## ResNet Transfer Learning

In [ ]:
# Fully Connected layer 부분을 제거하는 역할
resnet = ResNet50(weights='imagenet', include_top=False)

# ResNet50 레이어의 파라미터를 학습하지 않도록 설정
# 역전파를 통해 오차 정보가 전파 되더라도 파라미터가 업데이트되지 않는다.
for layer in resnet.layers:
    layer.trainable = False

# Fully connected layer 추가
x = resnet.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(7, activation='sigmoid')(x)
model3 = Model(resnet.input, predictions)

In [ ]:
model3.summary()

In [ ]:
model3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model3.fit(tr, validation_data=val, epochs=20)

## 데이터 추가 수집 및 하이퍼파라미터 튜닝

In [ ]:
tr = '/content/drive/MyDrive/style/tr'
val = '/content/drive/MyDrive/style/val'

tr_for_man = '/content/drive/MyDrive/style-for-man/tr'
val_for_man = '/content/drive/MyDrive/style-for-man/val'

# 클래스 리스트 선언
class_list = ['street', 'simple', 'classic', 'work', 'unique', 'sexy', 'girlish']
class_list_for_man = ['street', 'simple', 'classic', 'work', 'unique']

# woman
tr = tf.keras.preprocessing.image_dataset_from_directory(
    tr,
    labels="inferred",
    label_mode="categorical",
    class_names=class_list,
    seed=6,
)

val = tf.keras.preprocessing.image_dataset_from_directory(
    val,
    labels="inferred",
    label_mode="categorical",
    class_names=class_list,
    seed=6,
)

# man
tr_for_man = tf.keras.preprocessing.image_dataset_from_directory(
    tr_for_man,
    labels="inferred",
    label_mode="categorical",
    class_names=class_list_for_man,
    seed=6,
)

val_for_man = tf.keras.preprocessing.image_dataset_from_directory(
    val_for_man,
    labels="inferred",
    label_mode="categorical",
    class_names=class_list_for_man,
    seed=6,
)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Fully Connected layer 부분을 제거하는 역할
resnet = ResNet50(weights='imagenet', include_top=False)

# ResNet50 레이어의 파라미터를 학습하지 않도록 설정
# 역전파를 통해 오차 정보가 전파 되더라도 파라미터가 업데이트되지 않는다.
for layer in resnet.layers:
    layer.trainable = False

# Fully connected layer 추가
x = resnet.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(7, activation='sigmoid')(x)
model = Model(resnet.input, predictions)

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(tr, validation_data=val, epochs=50)

In [ ]:
from keras.models import load_model
model3.save('model-for-female.h5')
model4.save('model-for-male.h5')